## **Breast mass detection**
##### *Advanced Image Analysis 2022-2023*
##### *Luisana Álvarez Monsalve, Clara Lisazo, Xavier Beltrán Urbano*

##### **Import libraries and set paths of images**

In [2]:
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans
from cv2 import ximgproc

In [3]:
path_images = "/Users/clara/Desktop/MAIA/AIA/dataset/images/"
path_gt = "/Users/clara/Desktop/MAIA/AIA/dataset/groundtruth/"

##### **Functions**

In [4]:
def function_crop(downsampled_image, is_positive, img_gt=None):
    '''
    If image is negative crops the image to only contain the breast area.
    If image is positive crops both the image and the groundtruth.
    Args:
        downsampled_image (numpy.ndarray): is the downsampled breast image (either positive or negative)
        is_positive (bool): variable that determines if image is positive or negative
        img_gt (numpy.ndarray): groundtruth image. If the image is negative, no img_gt should be passed as argument, default is None.
    Outputs:
        For negative images:
            cropped_img (numpy.ndarray): cropped image
            None: because there is no groundtruth image to crop
        For positive images: 
            cropped_img (numpy.ndarray): cropped image
            cropped_gt (numpy.ndarray): cropped gt
    '''


    _ , imgBin = cv2.threshold(downsampled_image, 0 ,255, cv2.THRESH_BINARY)
    cnts, _ = cv2.findContours(imgBin.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(cnts, key = cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)
    cropped_img=downsampled_image[y:y+h, x:x+w]

    if is_positive:
       # Groundtruth should be cropped in the original scale, thus multiply the coordinates of bounding rectangle by the downsampling factor
       x_originalScale = x*4
       y_originalScale = y*4
       w_originalScale = w*4
       h_originalScale = h*4
       cropped_gt=img_gt[y_originalScale:y_originalScale+h_originalScale, x_originalScale:x_originalScale+w_originalScale]
       return cropped_img, cropped_gt
    else: 
       return cropped_img, None

In [5]:
def Clahe(image, tile_size, clip_limit):
    '''
    Computes the Contrast Limited Adaptive Histogram Equalization to an image.

    Args:
      image (numpy.ndarray): image to be processed
      tile_size (int)
      clip_limit (float)
    Outputs:
      CLAHE_image (numpy.ndarray): contrast enhanced image
    '''

    CLAHE = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(tile_size,tile_size))
    CLAHE_image = CLAHE.apply(image)
    return CLAHE_image

In [6]:
def MeanShift(image, sp, sr):
    '''
    Applies mean shift to input image.

    Args:
      image (numpy.ndarray): image to be processed
      sp (int): spatial window radius
      sr (int): color window radius 
    Output:
      meanShiftImage (numpy.ndarray): image with the applied meanshift
    '''
    # Convert to three-channel image in order to satisfy requirements of OpenCV's implementation of meanshift
    threeChannelImage = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    meanShiftImage = cv2.pyrMeanShiftFiltering(threeChannelImage, sp, sr, None, 1)
    meanShiftImage = cv2.cvtColor(meanShiftImage, cv2.COLOR_RGB2GRAY)

    return meanShiftImage

In [7]:
def MultiScaleMorphology(image, num_scales, size_opening, size_topHat):

    ''' 
    Applies grayscale morphology at different sizes of structuring elements

    Args:
        image (numpy.ndarray): preprocessed image
        num_scales (int): number of scales (sizes) at which the grayscale morphology will be applied
        size_opening (list): 1D vector of size equal to num_scales that specifies the radius of the disk-shaped 
            structuring element used for morphological opening
        size_topHat (list): 1D vector of size equal to num_scales that specifies the radius of the disk-shaped 
            structuring element used for morphological Top Hat
    Output:
        morphologyList (list): a list (size = num_scales) containing the resulting images after applying the grayscale morphology operations

    '''

    morphology_vector_topHat = [None] * num_scales
    morphology_vector_opening = [None] * num_scales
    morphologyList = [None] * num_scales


    for i in range(num_scales):

        # Apply Top Hat with structuring element ellipse of size size_topHat[i]
        structElemTopHat = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size_topHat[i], size_topHat[i]))
        morphology_vector_topHat[i] = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, structElemTopHat)
        
        # Apply opening to the result of Top Hat with structuring element ellipse of size size_opening[i]
        structElemOpening = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size_opening[i], size_opening[i]))
        morphology_vector_opening[i] = cv2.morphologyEx(morphology_vector_topHat[i], cv2.MORPH_OPEN, structElemOpening)
        
        # Linearly scale pixel values to 8-bit grayscale
        morphologyList[i] = np.zeros_like(morphology_vector_opening[i], dtype=np.uint8)
        cv2.normalize(morphology_vector_opening[i], morphologyList[i], 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    return morphologyList

In [8]:
def GaussianBlurring(list_imgs, kernel_size):
    ''' 
    Applies Gaussian blurring to all the images in list_imgs

    Args:
      list_imgs (list): list of images to which Gaussian blurring will be applied
      kernel_size (int): size of the Gaussian kernel

    Output:
      list_imgs_blurred (list): list of Gaussian blurred images
    '''
    list_imgs_blurred = []
    for i in range(len(list_imgs)):
      img_blurred = cv2.GaussianBlur(list_imgs[i], (kernel_size, kernel_size), 0, 0)
      list_imgs_blurred.append(img_blurred)

    return list_imgs_blurred


In [9]:
def Superpixel(list_imgs, num_scales, size_opening, num_iterations):
    '''
    Use Simple Linear Iterative Clustering (SLIC) for superpixel generation

    Args:
      list_imgs (list): list of images to which SLIC will be applied
      num_scales (int): number of scales
      size_opening (list): 1D vector of size equal to num_scales that specifies the radius of the disk-shaped 
            structuring element used for morphological opening
      num_iterations (int): number of iteratios for the SLIC algorithm

    Outputs:
      superpixeMeanslList (list): 1D vector of size equal to num_scales that contains the images with the mean intensities inside each superpixel
    '''
    
    vector_imgTesselated = [None] * num_scales
    superpixelMeansList = [None] * num_scales

    for i in range(num_scales):
      input = list_imgs[i]

      r = size_opening[i] 
      SLIC_algorithm = ximgproc.createSuperpixelSLIC(input, cv2.ximgproc.SLICO, r//4)
      SLIC_algorithm.iterate(num_iterations)

      # Get label contour mask
      mask = SLIC_algorithm.getLabelContourMask()

      # Tesselated image
      vector_imgTesselated[i] = cv2.cvtColor(input, cv2.COLOR_GRAY2BGR)
      vector_imgTesselated[i] = cv2.bitwise_and(vector_imgTesselated[i], vector_imgTesselated[i], mask=mask)
      
      labels = SLIC_algorithm.getLabels()

      # Replace original image pixels with superpixels means
      superpixelMeansList[i] = input.copy()

      for k in range(SLIC_algorithm.getNumberOfSuperpixels()):
          # Create a binary mask for the current superpixel label
          class_mask = cv2.inRange(labels, k, k)
          
          # Compute the mean color of the current superpixel and assign it to the output image
          mean_intensity = cv2.mean(input, mask=class_mask)[0]
          superpixelMeansList[i][class_mask > 0] = mean_intensity

    return superpixelMeansList

In [10]:
def Kmeans_Segmentation(list_imgs, k, num_scales):
    '''
    Performs k-means segmentation on all the images in list_imgs

    Args:
        list_imgs (list): list of images in which K-means segmentation will be applied
        k (int): number of clusters for K-means segmentation
        num_scales (int): number of scales

    Output:
        KMeansMaskList (list): 1D vector of size equal to num_scales that contains the binary images resulting of the K-means segmentation
    '''
    KMeansMaskList  = [None] * num_scales

    for i in range(len(list_imgs)):
        # Flatten the superpixel means into a 2D array
        imgClustered_flat = list_imgs[i].reshape((-1, 1))

        # Perform k-means clustering with k clusters
        kmeans = KMeans(n_clusters=k, random_state=0, n_init=10).fit(imgClustered_flat)

        # Reshape the cluster labels back into the original image shape
        cluster_labels = kmeans.labels_.reshape(list_imgs[i].shape)

        centroids = kmeans.cluster_centers_

        # Obtaining the binary mask that corresponds to the cluster with highest intensity
        KMeansMaskList[i] = np.zeros_like(list_imgs[i])
        KMeansMaskList[i][cluster_labels== np.argmax(centroids[:,0])] = 255

    return KMeansMaskList

In [11]:
def GT_Mask(cropped_gt):
    ''' 
    Finds the contours of the ground truth and creates a binary image per each mass

    Arg:
      cropped_gt (numpy.ndarray): cropped ground truth image

    Output:
      vector_gt_mask (list): list that contains one binary image per mass
      contours_gt (list): contours of the ground truth
    '''

    vector_gt_mask = []

    contours_gt, _ = cv2.findContours(cropped_gt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Create a binary mask per mass
    for i in range(len(contours_gt)):
      mask = np.zeros_like(cropped_gt)
      cv2.drawContours(mask, contours_gt, i, (255, 255, 255), cv2.FILLED)
      vector_gt_mask.append(mask)

    return vector_gt_mask,contours_gt

In [12]:
def IOU_calculation(vector_gt_mask,contours_vector,contours_gt):
  """
  Calculate the Intersection Over Union (IOU) between two binary images.

  Args:
      vector_gt_mask (list): list containing one binary mask per mass of the ground truth
      contours_vector (list): contous of the candidate regions
      contours_gt (list): contours of the ground truth

  Output:
      IOU_vector (list): list that contains the maximum IOU for each mass
  """

  IOU_vector = []

  for k in range(len(contours_gt)):
    max_iou = 0
    for contours in contours_vector:
      for contour in contours:
        mask_candidate = np.zeros_like(vector_gt_mask[k])
        cv2.drawContours(mask_candidate, [contour], 0, (255, 255, 255), cv2.FILLED)

        # Compute the intersection and union images
        intersection = cv2.bitwise_and(mask_candidate, vector_gt_mask[k])
        union = cv2.bitwise_or(mask_candidate, vector_gt_mask[k])

        # Compute the number of white pixels in the intersection and union images
        intersection_pixels = cv2.countNonZero(intersection)
        union_pixels = cv2.countNonZero(union)

        # Compute the IoU
        iou = intersection_pixels / union_pixels

        if iou > max_iou:
          max_iou = iou

    IOU_vector.append(max_iou)

    return IOU_vector

##### **Main code**

In [13]:
# Sort the files in the directory in which we will iterate:
dir_images_sorted = sorted(os.listdir(path_images))

# Define an empty vector in which we will store the IOU per mass in the positive images
IOU_vector_final=[]

# Number of scales that will be used in the multi-scale grayscale morphology
num_scales = 11

# Sizes of structuring elements that will be used for multi-scale grayscale morphology
size_opening = [8, 16, 24, 32, 40, 48, 56, 64, 72, 92, 120]
size_topHat = [16, 32, 48, 64, 80, 96, 112, 128, 144, 184, 240]

# Maximum and minimum areas for filtering the contours in the segmented image
min_area = 2000
max_area = 800000

# Main loop in which we iterate for all the images (positive and negative):
for index, name_image in enumerate(dir_images_sorted): 
  # To override undesired files:
  if name_image!="Thumbs.db" and name_image!=".DS_Store":
    print('Processing image number: ', index)
    img_breast = cv2.imread(path_images +  name_image, cv2.IMREAD_GRAYSCALE)
    # Create a boolean dummy variable to check if image is positive or negative:
    if not os.path.exists(os.path.join(path_gt, name_image)):
        is_positive = False
    else:
        is_positive = True
        img_gt = cv2.imread(path_gt +  name_image, cv2.IMREAD_GRAYSCALE)

    ##################### PREPROCESSING #####################
    # Downsampling the breast image in a factor of 4:
    downsampled_img = cv2.resize(img_breast, (img_breast.shape[1] // 4, img_breast.shape[0] // 4), interpolation=cv2.INTER_CUBIC)
    # Cropping the image to keep only the breast (remove part of the background):
    cropped_img,cropped_gt=function_crop(downsampled_img, is_positive, img_gt)
    # Apply CLAHE to the cropped downsampled image:
    CLAHE_img = Clahe(cropped_img, clip_limit=10.0, tile_size=6)
    # Mean shift filtering
    meanShiftImage = MeanShift(CLAHE_img, sp=5, sr=5)

    ##################### MULTI-SCALE GRAYSCALE MORPHOLOGY #####################
    # Apply grayscale morphology for different structuring element sizes (multiscale)
    morphologyList=MultiScaleMorphology(meanShiftImage, num_scales, size_opening, size_topHat)

    ##################### SEGMENTATION #####################
    # Apply Gaussian blurring to all the images in morphologyList
    imgsBlurredList = GaussianBlurring(morphologyList, kernel_size=3)
    # Superpixel generation
    superpixelsMeanList = Superpixel(imgsBlurredList, num_scales=num_scales, size_opening=size_opening, num_iterations=10)
    # K-means segmentation 
    KMeansMaskList = Kmeans_Segmentation(superpixelsMeanList, k=4, num_scales=num_scales)


    # Upsampling of the cropped original image
    originalUpsampled = cv2.resize(cropped_img, None, fx=4, fy=4, interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(f'/Users/clara/Desktop/MAIA/AIA/cropped_imgs_upsampled_all/{name_image}', originalUpsampled)

    # Upsample and find contours in the segmented images of all the scales
    segmentedUpsampledList = [None]*num_scales
    contours_vector = []  # vector that contains the contours of all the candidates of all the scales of one image
    for i in range(num_scales):
        segmentedUpsampledImage = cv2.resize(KMeansMaskList[i], None, fx=4, fy=4, interpolation=cv2.INTER_NEAREST)
        contours, _ = cv2.findContours(segmentedUpsampledImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        # Filter the contours by area
        filtered_contours = []
        for contour in contours:
            contour_area = cv2.contourArea(contour)
            if min_area <= contour_area <= max_area:
                filtered_contours.append(contour)
        contours_vector.append(filtered_contours)

        # Create a binary image with the segmentation of the scale i
        final_segmented_image = np.zeros_like(segmentedUpsampledImage)
        cv2.drawContours(final_segmented_image, filtered_contours, -1, (255, 255, 255), cv2.FILLED)
        cv2.imwrite(f'/Users/clara/Desktop/MAIA/AIA/segmented_images_new/{name_image[:-4]}_{i}.tif', final_segmented_image)

    if is_positive:
        cv2.imwrite(f'/Users/clara/Desktop/MAIA/AIA/cropped_gt_all/{name_image}', cropped_gt)
        # Obtain ground truth mask and contours
        vector_gt_mask,contours_gt=GT_Mask(cropped_gt)
        # Compute the IOU of the regions obtained with the segmentation with the ground truth regions
        IOU_vector=IOU_calculation(vector_gt_mask,contours_vector,contours_gt)
        IOU_vector_final.append(IOU_vector)

    

Processing image number:  0
Processing image number:  1
Processing image number:  2
Processing image number:  3
Processing image number:  4
Processing image number:  5
Processing image number:  6
Processing image number:  7
Processing image number:  8
Processing image number:  9
Processing image number:  10
Processing image number:  11
Processing image number:  12
Processing image number:  13
Processing image number:  14
Processing image number:  15
Processing image number:  16
Processing image number:  17
Processing image number:  18
Processing image number:  19
Processing image number:  20
Processing image number:  21
Processing image number:  22
Processing image number:  23
Processing image number:  24
Processing image number:  25
Processing image number:  26
Processing image number:  27
Processing image number:  28
Processing image number:  29
Processing image number:  30
Processing image number:  31
Processing image number:  32
Processing image number:  33
Processing image number:

In [15]:
# Calculate the sensitivity of the method
tp = 0
fn = 0
print(IOU_vector_final)
for i in range(len(IOU_vector_final)):
  if IOU_vector_final[i][0] > 0.2:
    tp = tp+1
  else:
    fn = fn+1
sensitivity = tp / (tp + fn)
print(sensitivity)

[[0.3226558748885383], [0.5072491846685395], [0.4045369794903667], [0.6570442286947141], [0.8294031799677395], [0.8507967998619886], [0.8284321272773929], [0.4516714174656647], [0.76287852013797], [0.8659996534162567], [0.5066008587135348], [0.3112340378687803], [0.824371520453681], [0.7096774193548387], [0.7366737114007279], [0.8240536005955622], [0.7586821844472653], [0.8769987746111457], [0.8561111100005797], [0.8270817165358857], [0.7673592799634357], [0.8314774234903031], [0.8907261831888914], [0.9044448959590768], [0.8701410987691384], [0.8015566550018101], [0.7961749940234282], [0.7797289642250207], [0.7409301486071836], [0.6414518565704128], [0.8415248157154558], [0.4657362428144777], [0.665372460496614], [0.6405810426018126], [0.8275173053593092], [0.8411445611680097], [0.7065168560579193], [0.6605054634328112], [0.7126762012552732], [0.8422300660693275], [0.7413704489687645], [0.7641313660161828], [0.6083427821209046], [0.9129595616847741], [0.8081130065660821], [0.8218060559